# COVID-19 - Bestätigte Fälle und Todesfälle - Weltweit

*Ein Projekt zur Erbringungen der portfoliorelevanten Leistung für den Kurs **Data Jornalism** im Modul **23-TXT-BaCL5** im Studiengang **Texttechnologie und Computerlinguistik** der **Universität Bielefeld**.*

# Requirements

Alle Programme, die zur Ausführung des Codes notwendig sind, befinden sich in der Textdatei `requirements.txt` und lassen sich über den folgenden Befehl per `pip3` installieren.

`pip3 install -r requirements.txt`

# Imports

Für die Ausführung des Projektes benötigen wir folgende Python-Pakete:

- `numpy` 
- `pandas`, um den Typ `DataFrame` zu nutzen und unsere tabellarischen Daten zu verarbeiten.
- `plotly`, um die Daten auf einer Landkarte darzustellen.
- `urllib.requests`, um die aktuellsten Zahlen herunterzuladen.
- `datetime`, um das heutige Datum herauszufinden.

In [22]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import urllib.request
from datetime import datetime, timedelta

# Datensatz

## Quelle

Der Datensatz ist den Zahlen des **ECDC - European Centre for Disease Prevention and Control** ([Website](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide)) entnommmen.

## Automatisierte Beschaffung

Die Daten befinden sich auf der Website des ECDC im `.xlsx`-Format. Zunächst wird die Datei heruntergeladen und im `assets/`-Ordner gesichert. Dabei versuchen wir, die Daten von Heute herunterzuladen. Sollten diese (noch) nicht verfügbar sein, werden die Daten vom gestrigen Tag heruntergeladen. Wenn auch diese nicht verfügbar sind, findet ein Fallback statt auf die letzte händisch heruntergeladene und geprüfte Datei vom `21.03.2020`.

Nach dem Versuch, die Tagesaktuellen Zahlen herunterzuladen, wird die Variable `data_file` auf die aktuellste Datei festgelegt.

In [23]:
url_base = 'https://www.ecdc.europa.eu/sites/default/files/documents/'
url_file = 'COVID-19-geographic-disbtribution-worldwide-{}.xlsx'

try:
    date = datetime.date(datetime.now())
    url_today = url_file.format(date)
    url = url_base + url_today
    
    urllib.request.urlretrieve(url, 'assets/' + url_today)
    url_file = url_today
except:
    try:
        date = date - timedelta(days=1)
        url_yesterday = url_file.format(date)
        url = url_base + url_yesterday

        urllib.request.urlretrieve(url, 'assets/' + url_yesterday)
        url_file = url_yesterday
    except:
        url_file = 'COVID-19-geographic-disbtribution-worldwide-2020-03-21.xlsx'
finally:
    data_file = url_file
    print('Genutzte Datei: ' + data_file)

Genutzte Datei: COVID-19-geographic-disbtribution-worldwide-2020-03-21.xlsx


Im Anschluss wird die Datei über `pandas` eingelesen und als `DataFrame`-Objekt gespeichert, damit wir die Daten tabellarisch auswerten können. Bevor wir Änderungen an den Daten vornehmen, werden wir diese in einer weiteren Variable zwischenspeichern. Auf diese Art und Weise können wir auch nach Veränderungen immer auf die Ausgangsdaten zurückgreifen, um etwaige Fehler zu finden.

Zum Test der Funktionalität, lassen wir uns die ersten Zeilen des entstandenen `DataFrame` ausgeben. Dies dient außerdem der Überprüfung, ob die Daten weiterer Bereinigung bedürfen.

In [24]:
data = pd.read_excel('assets/' + data_file)
data_raw = data
data.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId
0,2020-03-21,21,3,2020,2,0,Afghanistan,AF
1,2020-03-20,20,3,2020,0,0,Afghanistan,AF
2,2020-03-19,19,3,2020,0,0,Afghanistan,AF
3,2020-03-18,18,3,2020,1,0,Afghanistan,AF
4,2020-03-17,17,3,2020,5,0,Afghanistan,AF


## Sichtung

Die ersten Zeilen der Datei benötigen keine Bereinigung. Sowohl die Indizes als auch die Kopfzeile funktionieren wie gewünscht. Da wir eine Datumsspalte haben, werden die Spalten für Tag, Monat und Jahr prinzipiell nicht benötigt. Da diese im weiteren Verlauf jedoch nicht störend sind, können wir die Spalten so beibehalten.

Eine Fehlerquelle in den Daten kann eine fehlende oder falsch formatierte `GeoId` sein, weshalb wir uns die vorhandenen `GeoId`s ausgeben lassen. Da wir keine Duplikate haben wollen, nutzen wir für die Ausgabe eine Menge.

In [25]:
geoids = set()

for geoid in data['GeoId']:
    geoids.add(geoid)

print('Anzahl der verschiedenen Länder: ' + str(len(geoids)))
print(geoids)

Anzahl der verschiedenen Länder: 175
{nan, 'IQ', 'NE', 'PH', 'BE', 'BG', 'AE', 'NI', 'MG', 'UY', 'DZ', 'PT', 'BM', 'KR', 'US', 'NL', 'EG', 'LI', 'IM', 'CA', 'FO', 'DJ', 'RS', 'JPG11668', 'DK', 'KE', 'HR', 'NP', 'TR', 'TD', 'SK', 'CM', 'SM', 'BN', 'CV', 'SC', 'UZ', 'SV', 'CY', 'GA', 'SN', 'ZM', 'AZ', 'MM', 'LV', 'MN', 'MU', 'FI', 'LU', 'LC', 'MK', 'GQ', 'AD', 'TG', 'AG', 'IL', 'HN', 'PE', 'GT', 'SA', 'SE', 'VC', 'SI', 'TT', 'AL', 'CN', 'LB', 'PYF', 'PA', 'CH', 'IE', 'PG', 'ME', 'CI', 'IT', 'BY', 'TH', 'VN', 'ZW', 'JE', 'AM', 'KZ', 'PK', 'PY', 'BF', 'GI', 'MR', 'FJ', 'CU', 'KH', 'KW', 'BB', 'BA', 'QA', 'JO', 'RU', 'MV', 'GM', 'ID', 'GE', 'HT', 'EL', 'TZ', 'SR', 'HU', 'TW', 'MX', 'AR', 'ET', 'FR', 'NG', 'JP', 'TN', 'AF', 'CL', 'VA', 'PS', 'CR', 'BO', 'CZ', 'MT', 'SZ', 'AN', 'NO', 'OM', 'MS', 'MC', 'GH', 'DE', 'EC', 'MY', 'UK', 'GG', 'CG', 'VE', 'UA', 'CO', 'ES', 'LK', 'CD', 'SG', 'BH', 'BS', 'BT', 'BD', 'PL', 'SO', 'AU', 'GY', 'LT', 'GU', 'IN', 'NZ', 'RW', 'ZA', 'NC', 'SD', 'GL', 'BR', 'D

Hier fällt auf, dass einige Einträge vorhanden sind, die nicht über einen zweistelligen Länder-Code abgebildet werden. Die nicht-zweistelligen Ländercodes ermitteln wir wiefolgt, um herauszufinden, bei welchen Einträgen Reinigungsbedarf besteht.

In [26]:
geoid_error = data[data['GeoId'].str.len() != 2]

false_geoid = set()

for country in geoid_error['Countries and territories']:
    false_geoid.add(country)
    
for false_id in false_geoid:
    print(false_id)

Namibia
French_Polynesia
Cases_on_an_international_conveyance_Japan


Wirft man einen Blick auf diese Einträge, stellt man fest, dass für *Namibia* keine Einträge in der `GeoId` vorhanden sind (`nan`), *French_Polynesia* bereits einen dreistelligen Länder-Code eingetragen hat und *Cases_on_an_international_conveyance_Japan* eine spezielle achtstellige `GeoId` zugewiesen bekommen hat.

Über eine kurze Recherche lässt sich schnell herausfinden, dass der `Alpha-3`-Code für *Namibia* `NAM` ist. Bei *Cases_on_an_international_conveyance_Japan* handelt es sich um das Passagier-Schiff *Diamond Princess*, welches vor dem Hafen von Yokohama in Japan liegt/lag und in den Daten nicht zu Japans Fällen dazugezählt wird.

An dieser Stelle haben wir zwei Möglichkeiten, die Daten zu bereinigen, da `Plotly` zur Darstellung der Daten dreistellige Länder-Codes benötigt.

1. Die Fälle der *Diamond Princess* nicht auf der Weltkarte darstellen, also ein Sub-Set unserer Daten erstellen, aus dem wir diese herausnehmen
2. Die Fälle einem Land (z.B. Japan) zuordnen
3. Die Fälle weiterhin unter *Cases_on_an_international_conveyance_Japan* führen

Für das Erstellen der Weltkarte benötigen wir zwingend dreistellige Länder-Codes. Da die *Diamond Princess* nicht über einen solchen verfügt, ist für die Weltkarte die dritte Option nicht nützlich. Dennoch werden wir die Daten erhalten, um über Gesamt-Fälle Aussagen treffen zu können. Diese Daten werden weiter unter der Variable `data` geführt.

Für die Weltkarten nutzen wir Option 1 und bilden einen reduzierten Datensatz ohne die Fälle der *Diamond Princess*.

## Aufbereitung & Bereinigung

Für Namibia können wir bereits den entsprechenden Länder-Code in die `GeoId`-Spalte einfügen.

In [27]:
data.replace("nan", "NA", inplace=True)

Um die Daten im späteren Verlauf per `plotly` auf einer Weltkarte darstellen zu können, benötigen wir Länder-Codes im Format `ISO3166 Alpha-3`. Die `GeoId` aus den vorhanden Daten nutzt jedoch `ISO3166 Alpha-2`, weshalb wir eine weitere Spalte zu unseren Daten hinzufügen werden, die die entsprechenden Codes enthält. Hier bedienen wir uns einer Liste, die sowohl `Alpha-2`- als auch `Alpha-3`-Codes enthält.

In [28]:
iso3166 = pd.read_csv('assets/iso3166.csv')
iso3166.head()

,ISO3166-ALPHA-2,ISO3166-ALPHA-3
0,AF,AFG
1,AX,ALA
2,AL,ALB
3,AN,ANT
4,DZ,DZA


Nun können wir die Spalte mit den dreistelligen Länder-Codes hinzufügen. Dazu nutzen wir die vorher bereits importierte `ISO3166`-Liste.

In [29]:
data['GeoId3'] = data['GeoId'].replace(iso3166.set_index('ISO3166-ALPHA-2')['ISO3166-ALPHA-3'])
data.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,GeoId3
0,2020-03-21,21,3,2020,2,0,Afghanistan,AF,AFG
1,2020-03-20,20,3,2020,0,0,Afghanistan,AF,AFG
2,2020-03-19,19,3,2020,0,0,Afghanistan,AF,AFG
3,2020-03-18,18,3,2020,1,0,Afghanistan,AF,AFG
4,2020-03-17,17,3,2020,5,0,Afghanistan,AF,AFG


Wir wir hier sehen können, wurde die benötigte Spalte `GeoId3` hinzugefügt und mit den entsprechenden dreistelligen Länder-Codes befüllt.

Da sowohl die Index-Spalte als auch die Kopfzeile der Tabelle bereits vielversprechend formatiert sind und es keine weiteren Daten gibt, die bereinigt werden müssen, können wir den Datensatz so weiter verwenden. 

Zusätzlich werden wir ein Sub-Set der Daten erstellen, welches für die Erstellung der Weltkarten bereinigt wurde. Hier werden wir die Fälle *nicht* Japan zuordnen sondern die Fälle gänzlich aus den Daten streichen. Grund dafür ist, dass bei der Darstellung auf der Weltkarte für uns im Vordergrund steht, eine Fall-Zuweisung zu den Ländern zu erhalten.

In [30]:
data_map = data[data['Countries and territories'] != 'Cases_on_an_international_conveyance_Japan']

# Anwendung - Statistik

##  Tagesdaten für ein Land

Beispielsweise können wir die Daten in kleinere Einheiten aufteilen, um nicht zu jeder Zeit mit dem gesamten Datensatz arbeiten zu müssen. Über Ansprechen der `GeoId`-Spalte können wir die Daten für Deutschland hersausfiltern. Wir speichern die Daten entsprechend in der Variable `data_deu`.

In [31]:
def data_subset_country(geoid3):
    return data[data.GeoId3 == geoid3]

data_deu = data_subset_country("DEU")
data_deu.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,GeoId3
2131,2020-03-21,21,3,2020,4185,2,Germany,DE,DEU
2132,2020-03-20,20,3,2020,5940,30,Germany,DE,DEU
2133,2020-03-19,19,3,2020,1042,0,Germany,DE,DEU
2134,2020-03-18,18,3,2020,1144,0,Germany,DE,DEU
2135,2020-03-17,17,3,2020,1174,1,Germany,DE,DEU


Hier sieht man der Verlauf der neu bestätigten Fälle und Todesfälle in Deutschland.

In [32]:
def plot_verlauf(data):
    fig = make_subplots(rows=2, cols=1)

    fig.add_trace(
        go.Scatter(x=data.DateRep, y=data.Cases, name="Fälle"),
        row=1, col=1,
    )

    fig.add_trace(
        go.Scatter(x=data.DateRep, y=data.Deaths, name="Todesfälle"),
        row=2, col=1
    )
    
    fig.update_layout(title_text='Verlauf bestätigter Fälle und Todesfälle')

    fig.show()
    
plot_verlauf(data_deu)

Nun schauen wir, welche Daten ohne Weiteres aus der vorhandenen Tabell extrahiert werden können. Zum einen können wir die Summe der bestätigten Krankheitsfälle ausgeben lassen. Zum anderen lässt sich auch die Zahl der bestätigten Todesfälle extrahieren.

In [33]:
deu_cases = data_deu.Cases.sum()
deu_cases

18323

In [34]:
deu_deaths = data_deu.Deaths.sum()
deu_deaths

45

## Akkumulierte Daten

Um die Gesamtzahlen aller Fälle und Todesälle darstellen zu können, summieren wir die Tagesdaten auf und erhalten so eine Liste aller Länder und den aufsummierten Zahlen.

In [35]:
data_accumulated = data.groupby(['GeoId3', 
                                 "Countries and territories"], 
                                as_index=False).sum()[['GeoId3', 
                                                       'Countries and territories', 
                                                       'Cases', 
                                                       'Deaths']]
data_accumulated.head()

,GeoId3,Countries and territories,Cases,Deaths
0,AFG,Afghanistan,24,0
1,ALB,Albania,70,2
2,AND,Andorra,75,0
3,ANT,Netherlands_Antilles,13,0
4,ARE,United_Arab_Emirates,140,0


Diese lassen sich jetzt in einem Balkendiagramm plotten. Für die bessere Übersicht plotten wir hier die Infizierten-Fälle und die Todesfälle in einem Balkendiagramm.

In [36]:
def plot_gesamtzahlen(data=data_accumulated, sort=True, limit=True):
    """
    Horizontales Balkendiagramm mit den Gesamtzahlen der Fälle und Todesfälle.
    
    data: Datensatz
    sort: Boolean; soll sortiert werden?
    limit: Boolean; sollen nur die 'top' Einträge angezeigt werden? (Nur in Verbindung mit sort zu verwenden)
    """
    
    if sort:
        if limit:
            data = data.sort_values('Cases').tail(15)
        else:
            data = data.sort_values('Cases')
            
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=data['Cases'],
        y=data['Countries and territories'],
        orientation='h',
        name='Infizierte',
        marker_color="green"))
    
    fig.add_trace(go.Bar(
        x=data['Deaths'],
        y=data['Countries and territories'],
        orientation='h',
        name='Todesfälle',
        marker_color="red"))
     
    fig.update_layout(barmode='stack', xaxis_tickangle=-45, title_text='Bestätigte Fälle und Todesfälle')

    fig.show()

In [37]:
plot_gesamtzahlen(data_accumulated)

# Anwendung Weltkarten

## Fälle und Todesälle

In [38]:
def plot_world_map(data=data, plot_value='Cases'):
    """
    data: Datensatz, der geplottet werden soll
    plot_value: 'Cases' oder 'Deaths'
    """
    
    world_map = go.Figure(data=go.Choropleth(
        z = data[plot_value],
        locations = data['GeoId3'],
        colorscale = 'Greens' if plot_value == 'Cases' else 'Reds',
        marker_line_width=0.2,
        colorbar_title = plot_value,
        text = data['Countries and territories'],
    ))
    
    world_map.update_layout(geo=dict(showframe=False, showcoastlines=False, projection_type='equirectangular'))
    world_map.show()

## Bestätigte Infizierte

In [41]:
plot_world_map(data_accumulated, 'Cases')

## Bestätige Todesfälle

In [44]:
plot_world_map(data_accumulated, 'Deaths')

# Ausblick

Möglichkeiten, das Projekt zu erweitern:

- *Timeline*: Weltkarte mit Zeitleiste, um den Verlauf der bestätigten Fälle beobachten zu können. Dies ist jedoch mit den genutzten Programmen nicht möglich. Plotly bietet keine Zeitleisten für Weltkarten an. Sollte sich ein entsprechendes Programm finden, müsste der Datensatz so bearbeitet werden, dass für jeden Tag ein Eintrag für jedes Land existiert. Außerdem müssen die Zeilen im Datensatz immer die akkumulierten Zahlen zeigen; nicht die *an dem Tag* bestätigten.
- *Weitere Datensätze*: Der Datensatz des ECDC beschränkt sich auf wesentliche geografische Informationen. Eine Aufschlüsselung in kleinere geografische Einheiten war mir nicht möglich. Das Robert-Koch-Institut besitzt solche Daten, macht diese jedoch nicht für die Allgemeinheit zugänglich. Außerdem wäre interessant, ein Datensatz zu nutzen, der über die geografischen Daten hinaus auch Personendaten umfasst. Dabei ist das Alter der PatientInnen vermutlich besonders interessant.